First import the libraries that we need.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

The data of in and out of station passenger number of each MRT(Mass Rapid Transit) station in the greater Taipei area is available from the government, so let's make good use of it!

We download the compressed file from http://163.29.157.32:8080/dataset/98d67c29-464a-4003-9f78-b1cbb89bff59, and import it into csv format.

In [2]:
MRT_rawdata = pd.read_csv('MRT_data_201911.csv')

In [3]:
MRT_rawdata.head(10)

,日期 時段 進站 出站 人次
0,---------- ------ ----------------------------...
1,2019-11-01 00 松山機場 ...
2,2019-11-01 00 松山機場 ...
3,2019-11-01 00 松山機場 ...
4,2019-11-01 00 松山機場 ...
5,2019-11-01 00 松山機場 ...
6,2019-11-01 00 松山機場 ...
7,2019-11-01 00 松山機場 ...
8,2019-11-01 00 松山機場 ...
9,2019-11-01 00 松山機場 ...


It seems a bit odd when we look at the first 10 row of this dataset, let's try to figure out what is wrong.

In [4]:
MRT_rawdata.shape

(7348323, 1)

In [5]:
MRT_rawdata.iat[5, 0]

'2019-11-01 00     松山機場                           大安                             0'

It seems that all data are clustered into the first column, separated by a tab or several spaces.

Let's restructure it by putting it into a list.

In [6]:
MRT_rawdata_aslist = []

for i in range(MRT_rawdata.shape[0]):
    line = ' '
    line = MRT_rawdata.iat[i,0]
    MRT_rawdata_aslist.append(line.split())

MRT_rawdata_aslist[:10]

[['----------',
  '------',
  '------------------------------',
  '------------------------------',
  '-----------'],
 ['2019-11-01', '00', '松山機場', '松山機場', '0'],
 ['2019-11-01', '00', '松山機場', '中山國中', '0'],
 ['2019-11-01', '00', '松山機場', '南京復興', '0'],
 ['2019-11-01', '00', '松山機場', '忠孝復興', '0'],
 ['2019-11-01', '00', '松山機場', '大安', '0'],
 ['2019-11-01', '00', '松山機場', '科技大樓', '0'],
 ['2019-11-01', '00', '松山機場', '六張犁', '0'],
 ['2019-11-01', '00', '松山機場', '麟光', '0'],
 ['2019-11-01', '00', '松山機場', '辛亥', '0']]

From above we know that the first row contains things we don't need. Let's fix it by dropping it, and also do the same examination on the tail part of this list.

In [7]:
MRT_rawdata_aslist = MRT_rawdata_aslist[1:]
MRT_rawdata_aslist[:10]

[['2019-11-01', '00', '松山機場', '松山機場', '0'],
 ['2019-11-01', '00', '松山機場', '中山國中', '0'],
 ['2019-11-01', '00', '松山機場', '南京復興', '0'],
 ['2019-11-01', '00', '松山機場', '忠孝復興', '0'],
 ['2019-11-01', '00', '松山機場', '大安', '0'],
 ['2019-11-01', '00', '松山機場', '科技大樓', '0'],
 ['2019-11-01', '00', '松山機場', '六張犁', '0'],
 ['2019-11-01', '00', '松山機場', '麟光', '0'],
 ['2019-11-01', '00', '松山機場', '辛亥', '0'],
 ['2019-11-01', '00', '松山機場', '萬芳醫院', '0']]

In [8]:
MRT_rawdata_aslist[-10:]

[['2019-11-30', '23', '丹鳳', '東門', '0'],
 ['2019-11-30', '23', '丹鳳', '蘆洲', '1'],
 ['2019-11-30', '23', '丹鳳', '三民高中', '0'],
 ['2019-11-30', '23', '丹鳳', '徐匯中學', '4'],
 ['2019-11-30', '23', '丹鳳', '三和國中', '2'],
 ['2019-11-30', '23', '丹鳳', '三重國小', '0'],
 ['2019-11-30', '23', '丹鳳', '迴龍', '3'],
 ['2019-11-30', '23', '丹鳳', '丹鳳', '8'],
 ['警告:', '彙總或其他', 'SET', '作業已刪除', 'Null', '值。'],
 ['(7348320', '個資料列受到影響)']]

In [9]:
MRT_rawdata_aslist = MRT_rawdata_aslist[:-2]
MRT_rawdata_aslist[-10:]

[['2019-11-30', '23', '丹鳳', '中山國小', '1'],
 ['2019-11-30', '23', '丹鳳', '行天宮', '5'],
 ['2019-11-30', '23', '丹鳳', '東門', '0'],
 ['2019-11-30', '23', '丹鳳', '蘆洲', '1'],
 ['2019-11-30', '23', '丹鳳', '三民高中', '0'],
 ['2019-11-30', '23', '丹鳳', '徐匯中學', '4'],
 ['2019-11-30', '23', '丹鳳', '三和國中', '2'],
 ['2019-11-30', '23', '丹鳳', '三重國小', '0'],
 ['2019-11-30', '23', '丹鳳', '迴龍', '3'],
 ['2019-11-30', '23', '丹鳳', '丹鳳', '8']]

We can see that the list has 5 columns, each representing **date, hour, station(in), station(out), and number of passengers**.

The goal of using this dataset is to understand which MRT stations are packed with office workers. Since we are not interested in any particular date, we integrate the date/hour and change it to weekly hours(Monday 11 PM will be 23, and Wednesday 10 AM will be 48+10=58, ...etc) to better understand the passenger flow.

In [10]:
import datetime

MRT_rawdata_aslist_weekly = []

for x in range(len(MRT_rawdata_aslist)):
    year = int(MRT_rawdata_aslist[x][0].split('-')[0])
    month = int(MRT_rawdata_aslist[x][0].split('-')[1])
    day = int(MRT_rawdata_aslist[x][0].split('-')[2])
    weekday = datetime.datetime(year, month, day).weekday()
    MRT_rawdata_aslist_weekly.append([0, 0, 0, 0])
    MRT_rawdata_aslist_weekly[x][0] = (weekday*24 + int(MRT_rawdata_aslist[x][1]))
    for i in range(3):
        MRT_rawdata_aslist_weekly[x][i+1] = (MRT_rawdata_aslist[x][i+2])
    
    
MRT_rawdata_aslist_weekly[:10]

[[96, '松山機場', '松山機場', '0'],
 [96, '松山機場', '中山國中', '0'],
 [96, '松山機場', '南京復興', '0'],
 [96, '松山機場', '忠孝復興', '0'],
 [96, '松山機場', '大安', '0'],
 [96, '松山機場', '科技大樓', '0'],
 [96, '松山機場', '六張犁', '0'],
 [96, '松山機場', '麟光', '0'],
 [96, '松山機場', '辛亥', '0'],
 [96, '松山機場', '萬芳醫院', '0']]

In order to make it understandable for people that doesn't speak Mandarian, we'll convert the station name into the matching station code.

So first we need to import a new data set that has both the Mandarian version of the station name and the station code.

In [16]:
MRT_geoloc = pd.read_csv('https://raw.githubusercontent.com/repeat/northern-taiwan-metro-stations/master/northern-taiwan.csv')
MRT_geoloc.head(10)

,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,address,lat,lon
0,BR01,BR13,動物園,Taipei Zoo,BR,文湖線,台北市文山區新光路二段32號,24.998197,121.579338
1,BR02,BR12,木柵,Muzha,BR,文湖線,台北市文山區木柵路四段135號,24.998241,121.573145
2,BR03,BR11,萬芳社區,Wanfang Community,BR,文湖線,台北市文山區萬芳路60號,24.998585,121.568102
3,BR04,BR10,萬芳醫院,Wanfang Hospital,BR,文湖線,台北市文山區興隆路三段113號,24.999386,121.558152
4,BR05,BR9,辛亥,Xinhai,BR,文湖線,台北市文山區辛亥路四段128號,25.005475,121.557107
5,BR06,BR8,麟光,Linguang,BR,文湖線,台北市大安區和平東路三段410號,25.018535,121.558791
6,BR07,BR7,六張犁,Liuzhangli,BR,文湖線,台北市大安區和平東路三段168號,25.023777,121.553115
7,BR08,BR6,科技大樓,Technology Building,BR,文湖線,台北市大安區復興南路二段235號,25.026125,121.543437
8,BR09,BR5,大安,Daan,BR,文湖線,台北市大安區信義路四段2號,25.032943,121.543551
9,BR10,BR4,忠孝復興,Zhongxiao Fuxing,BR,文湖線,台北市大安區忠孝東路四段47號,25.041629,121.543767


There are stations that have 2 station codes since they are transfer stations that exist in different MRT lines. Let's keep the first station code and drop the second. Also, we want to create a list that contains the Mandarian station name and another list that contains the station code, with both indicating the same station at the same position in the list.

In [17]:
MRT_name_list = []
MRT_stationcode_list = []

for i in range(MRT_geoloc.shape[0]):
    x = MRT_geoloc.at[i, 'station_name_tw']
    if x not in MRT_name_list:
        MRT_name_list.append(x)
        MRT_stationcode_list.append(MRT_geoloc.at[i, 'station_code'])
    else:
        MRT_geoloc = MRT_geoloc.drop(index = i)
        
x = MRT_name_list.index('大橋頭')
MRT_name_list[x] = '大橋頭站'

In [20]:
MRT_name_list[:10]

['動物園', '木柵', '萬芳社區', '萬芳醫院', '辛亥', '麟光', '六張犁', '科技大樓', '大安', '忠孝復興']

In [21]:
MRT_stationcode_list[:10]

['BR01',
 'BR02',
 'BR03',
 'BR04',
 'BR05',
 'BR06',
 'BR07',
 'BR08',
 'BR09',
 'BR10']

Now we can get back and convert the station name into station code in the MRT_rawdata_aslist_weekly list.

In [22]:
for i in range(len(MRT_rawdata_aslist_weekly)):
    x = MRT_rawdata_aslist_weekly[i][1]
    x1 = MRT_name_list.index(x)
    MRT_rawdata_aslist_weekly[i][1] = MRT_stationcode_list[x1]
    y = MRT_rawdata_aslist_weekly[i][2]
    y1 = MRT_name_list.index(y)
    MRT_rawdata_aslist_weekly[i][2] = MRT_stationcode_list[y1]
    
MRT_rawdata_aslist_weekly[:10]

[[96, 'BR13', 'BR13', '0'],
 [96, 'BR13', 'BR12', '0'],
 [96, 'BR13', 'BR11', '0'],
 [96, 'BR13', 'BR10', '0'],
 [96, 'BR13', 'BR09', '0'],
 [96, 'BR13', 'BR08', '0'],
 [96, 'BR13', 'BR07', '0'],
 [96, 'BR13', 'BR06', '0'],
 [96, 'BR13', 'BR05', '0'],
 [96, 'BR13', 'BR04', '0']]

We'll add the heading and convert this list into a dataframe.

In [23]:
headings = ['weekly_hour', 'in_station', 'out_station', 'passenger_num']
MRT_df = pd.DataFrame(data = MRT_rawdata_aslist_weekly, columns = headings)
MRT_df.head(10)

,weekly_hour,in_station,out_station,passenger_num
0,96,BR13,BR13,0
1,96,BR13,BR12,0
2,96,BR13,BR11,0
3,96,BR13,BR10,0
4,96,BR13,BR09,0
5,96,BR13,BR08,0
6,96,BR13,BR07,0
7,96,BR13,BR06,0
8,96,BR13,BR05,0
9,96,BR13,BR04,0


It's time to group the data into a pivot table to visualize the passenger number in each weekly hour, separated by different stations(in). Before that, we also have to make sure the data type of passenger_num is float for arithmetic calculations.

In [24]:
MRT_df.dtypes

weekly_hour       int64
in_station       object
out_station      object
passenger_num    object
dtype: object

In [25]:
MRT_df = MRT_df.astype({'passenger_num': 'float'})

In [26]:
MRT_df_instation = MRT_df.pivot_table(index = ['in_station'], columns = 'weekly_hour', values = 'passenger_num', dropna = True, aggfunc = 'sum')
MRT_df_instation.head(10)

weekly_hour,0,1,5,6,7,8,9,10,11,12,...,158,159,160,161,162,163,164,165,166,167
in_station,,,,,,,,,,,,,,,,,,,,,
BL01,36.0,0.0,0.0,1342.0,4392.0,7947.0,3479.0,1653.0,1420.0,1250.0,...,1822.0,1618.0,1769.0,2133.0,1908.0,1211.0,857.0,517.0,331.0,193.0
BL02,84.0,0.0,0.0,1471.0,6043.0,10133.0,7248.0,3760.0,2871.0,2444.0,...,3229.0,2771.0,2961.0,3471.0,3323.0,2473.0,2292.0,2050.0,1146.0,567.0
BL03,54.0,2.0,0.0,996.0,3906.0,6726.0,3398.0,2052.0,1507.0,1337.0,...,1962.0,1540.0,1446.0,1721.0,1315.0,931.0,750.0,580.0,374.0,220.0
BL04,193.0,3.0,0.0,3081.0,11823.0,22256.0,10325.0,5578.0,3933.0,3718.0,...,5485.0,4155.0,3690.0,4498.0,3872.0,2819.0,2115.0,1574.0,1078.0,570.0
BL05,216.0,6.0,3.0,3054.0,12898.0,23064.0,11397.0,6512.0,5276.0,4960.0,...,4770.0,4246.0,4072.0,4991.0,3545.0,2354.0,2019.0,1594.0,1189.0,609.0
BL06,421.0,4.0,2.0,3015.0,13193.0,24250.0,13239.0,8001.0,6038.0,6038.0,...,8535.0,7440.0,7837.0,9247.0,7507.0,6017.0,5491.0,5233.0,3855.0,1592.0
BL07,807.0,12.0,3.0,2314.0,11471.0,25111.0,13874.0,8449.0,6900.0,7336.0,...,11960.0,11760.0,12211.0,14282.0,16006.0,15555.0,15790.0,15023.0,11888.0,5091.0
BL08,366.0,12.0,2.0,4587.0,19418.0,36868.0,17948.0,9893.0,7358.0,7260.0,...,10760.0,8691.0,7827.0,9137.0,8007.0,5883.0,4947.0,4085.0,2729.0,1321.0
BL09,271.0,4.0,1.0,2982.0,13214.0,22132.0,11368.0,5910.0,4470.0,4511.0,...,6119.0,5281.0,5450.0,6171.0,4962.0,3407.0,2846.0,2329.0,1659.0,800.0


In [27]:
MRT_df_instation.shape

(108, 147)

From the chart above we can see that there are very little passengers on weekly_hour 1 and 5, which correspond to Monday 1 and 5 AM. This totally makes sense since the MRT service starts at around 6 AM and ends at around 1 AM. For the data continuity, we'll also add the data of 2-4 AM each day into the dataframe.

In [28]:
for i in range(7):
    MRT_df_instation.insert(2 + i*24, 2 + i*24, float(0.0), True)
    MRT_df_instation.insert(3 + i*24, 3 + i*24, float(0.0), True) 
    MRT_df_instation.insert(4 + i*24, 4 + i*24, float(0.0), True)
MRT_df_instation.head(10)

weekly_hour,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
in_station,,,,,,,,,,,,,,,,,,,,,
BL01,36.0,0.0,0.0,0.0,0.0,0.0,1342.0,4392.0,7947.0,3479.0,...,1822.0,1618.0,1769.0,2133.0,1908.0,1211.0,857.0,517.0,331.0,193.0
BL02,84.0,0.0,0.0,0.0,0.0,0.0,1471.0,6043.0,10133.0,7248.0,...,3229.0,2771.0,2961.0,3471.0,3323.0,2473.0,2292.0,2050.0,1146.0,567.0
BL03,54.0,2.0,0.0,0.0,0.0,0.0,996.0,3906.0,6726.0,3398.0,...,1962.0,1540.0,1446.0,1721.0,1315.0,931.0,750.0,580.0,374.0,220.0
BL04,193.0,3.0,0.0,0.0,0.0,0.0,3081.0,11823.0,22256.0,10325.0,...,5485.0,4155.0,3690.0,4498.0,3872.0,2819.0,2115.0,1574.0,1078.0,570.0
BL05,216.0,6.0,0.0,0.0,0.0,3.0,3054.0,12898.0,23064.0,11397.0,...,4770.0,4246.0,4072.0,4991.0,3545.0,2354.0,2019.0,1594.0,1189.0,609.0
BL06,421.0,4.0,0.0,0.0,0.0,2.0,3015.0,13193.0,24250.0,13239.0,...,8535.0,7440.0,7837.0,9247.0,7507.0,6017.0,5491.0,5233.0,3855.0,1592.0
BL07,807.0,12.0,0.0,0.0,0.0,3.0,2314.0,11471.0,25111.0,13874.0,...,11960.0,11760.0,12211.0,14282.0,16006.0,15555.0,15790.0,15023.0,11888.0,5091.0
BL08,366.0,12.0,0.0,0.0,0.0,2.0,4587.0,19418.0,36868.0,17948.0,...,10760.0,8691.0,7827.0,9137.0,8007.0,5883.0,4947.0,4085.0,2729.0,1321.0
BL09,271.0,4.0,0.0,0.0,0.0,1.0,2982.0,13214.0,22132.0,11368.0,...,6119.0,5281.0,5450.0,6171.0,4962.0,3407.0,2846.0,2329.0,1659.0,800.0


Now we have 168 columns corresponding to 168 hours weekly. Hooray!!

We also want to process and get another dataframe that groups the data by station(out).

In [29]:
MRT_df_outstation = MRT_df.pivot_table(index = ['out_station'], columns = 'weekly_hour', values = 'passenger_num', dropna = True, aggfunc = 'sum')

for i in range(7):
    MRT_df_outstation.insert(2 + i*24, 2 + i*24, float(0.0), True)
    MRT_df_outstation.insert(3 + i*24, 3 + i*24, float(0.0), True) 
    MRT_df_outstation.insert(4 + i*24, 4 + i*24, float(0.0), True)
    
MRT_df_outstation.head(10)

weekly_hour,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
out_station,,,,,,,,,,,,,,,,,,,,,
BL01,434.0,24.0,0.0,0.0,0.0,0.0,645.0,4587.0,4021.0,956.0,...,1595.0,1649.0,1957.0,2300.0,2624.0,2132.0,2559.0,2338.0,2117.0,1238.0
BL02,259.0,17.0,0.0,0.0,0.0,0.0,1333.0,3156.0,3862.0,2159.0,...,2541.0,3084.0,3288.0,4692.0,4016.0,4305.0,3786.0,3419.0,2710.0,1483.0
BL03,339.0,25.0,0.0,0.0,0.0,0.0,440.0,1536.0,1412.0,781.0,...,1458.0,1456.0,1793.0,2379.0,2294.0,1995.0,1840.0,1949.0,1760.0,844.0
BL04,970.0,55.0,0.0,0.0,0.0,0.0,1181.0,5065.0,3211.0,1937.0,...,3716.0,3885.0,5746.0,6995.0,7514.0,5918.0,6260.0,6294.0,5855.0,2664.0
BL05,729.0,16.0,0.0,0.0,0.0,3.0,1364.0,5158.0,6949.0,6100.0,...,3411.0,3808.0,4420.0,5667.0,5822.0,5143.0,4780.0,4960.0,4243.0,1935.0
BL06,1000.0,32.0,0.0,0.0,0.0,2.0,899.0,2955.0,3817.0,3263.0,...,6785.0,6743.0,7823.0,10694.0,10744.0,8007.0,6721.0,6628.0,5607.0,2481.0
BL07,888.0,0.0,0.0,0.0,0.0,3.0,3362.0,8388.0,13909.0,6786.0,...,12978.0,14444.0,17989.0,24388.0,22404.0,15931.0,12430.0,9338.0,6752.0,2613.0
BL08,1144.0,0.0,0.0,0.0,0.0,2.0,1950.0,5790.0,8901.0,6306.0,...,6888.0,7975.0,9620.0,13392.0,12347.0,9916.0,9386.0,9887.0,8403.0,3481.0
BL09,829.0,0.0,0.0,0.0,0.0,1.0,660.0,2693.0,5871.0,3917.0,...,4984.0,5375.0,6445.0,8803.0,8249.0,6647.0,6519.0,6404.0,5495.0,2551.0


Now let's export these 2 datasets for future usage!

In [30]:
MRT_df_instation.to_csv('MRT_df_instation.csv', index = True)

In [31]:
MRT_df_outstation.to_csv('MRT_df_outstation.csv', index = True)